# Laboratorium 2
## Marcin Biela, gr1b, środa 14:50 </br></br>


Celem zajęć jest zapoznanie się z protokołem TIER a także poznanie pojęcia "Tidy Data", zastosowaniu, zasad obowiązujących oraz czynności które powinniśmy wykonać przy porządkowaniu danych.

W poniższej komórce znajdują się importy potrzebnych bibliotek do przetworzenia zbioru danych.


Zbiór którym będę się zajmował to zbiór numer 1: "weather.txt"

In [1]:
# Potrzebne importy
import pandas as pd
import os
import re
import numpy as np

In [2]:
# odczytuje scieżke do oryginalnego pliku w celu wczytania datasetu
original_data_path = os.getcwd()[:-13] + "original_data/weather1.txt"
print(original_data_path)

/home/mbiela/Documents/AiBD/lab2/original_data/weather1.txt


#### Poniższe 2 komórki mają za zadanie wczytać nasz dataset do pliku a następnie wstępnie go przefiltrować pobierając jedynie interesujący nas zakres danych czyli pierwsze 5 miesięcu roku 2010. 

In [3]:
with open(original_data_path) as data:
    file = data.readlines()

In [4]:
data2010 = []
for elem in file:
    if "2010" in elem[0:21]:
        searched_month = elem[15:17]
        if searched_month == '01' or searched_month == '02' or searched_month == '03' or searched_month == '04' or searched_month == '05':
            data2010.append(elem)

#### Poniższa komórka odpowiedzialna za wyczyszczenie zbioru danych z nieprawidłowych danych. </br></br>
- W tym celu korzystam już z przefiltrowanego wcześniej zbioru zapisanego w liście o nazwie data2010
- Usuwam wszyskie niepoprawne dane tj:</t> S, S-9999, -9999
- Flaga S wg dokumentacji z Global Historical Climatology Network oznacza 'failed spatial consistency check' zatem każda komórka zawierająca w sobie tą daną jest bezużyteczna i należy ją usunąć. 

In [5]:
col_names = ["id", "year", "month", "element"] + ["day{}".format(i) for i in range(1, 32)]
stationID = data2010[0][0:11]
# print(stationID)
generate_rows = []

for row in data2010:
    it = len(stationID)
    year = row[it:it+4]
    month = row[it+4:it+4+2]
    element_type = row[it+4+2:it+4+2+4]
    days_1 = row[it+4+2+4:].split()
    day_data = row[it+4+2+4:]
    
    # Z racji, że podczas dzielenia fragmentu zawierającego
    # temperatury między kolejnymi pomiarami występowały niejednostajne odstępy między próbkami
    # należy ograniczyć dane do 31 próbek.
    if len(days_1) > 31:
        days_1 = days_1[:31]

    to_return = []
    for el in days_1:
        # Usuwanie artefaktów
        if el == 'S-9999' or el == 'S' or el == '-9999':
            el = 'inf'
            to_return.append(el)
        else:
            to_return.append(el)
    
    # Dane zwrócone w postaci List[List] później z tej listy będzie tworzony 
    # pierwszy niewyczyszczony jeszcze DataFrame
    generate_rows.append([stationID, year, month, element_type] + to_return)

In [6]:
print(generate_rows)

[['MX000017004', '2010', '01', 'TMAX', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', '278', 'inf'], ['MX000017004', '2010', '01', 'TMIN', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', '145', 'inf'], ['MX000017004', '2010', '01', 'PRCP', 'inf', 'inf', 'inf', 'inf', '0', 'inf', '0', 'inf', 'inf', 'inf', 'inf', '0', 'inf', '0', 'inf', '0', 'inf', 'inf', '0', 'inf', '0', 'inf', '0', 'inf', '0', 'inf', '0', 'inf', '0', 'inf', '0'], ['MX000017004', '2010', '02', 'TMAX', 'inf', '273', 'inf', '241', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', '297', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', '299', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf', 'inf'],

In [7]:
# Stworzenie pierwszego DataFramu jeszcze nie zgodnego z zasadami tidy_data
df_not_tidied = pd.DataFrame(generate_rows, columns=col_names)
df_not_tidied.head(20)

,id,year,month,element,day1,day2,day3,day4,day5,day6,...,day22,day23,day24,day25,day26,day27,day28,day29,day30,day31
0,MX000017004,2010,01,TMAX,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,278,inf
1,MX000017004,2010,01,TMIN,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,145,inf
2,MX000017004,2010,01,PRCP,inf,inf,inf,inf,0,inf,...,inf,0,inf,0,inf,0,inf,0,inf,0
3,MX000017004,2010,02,TMAX,inf,273,inf,241,inf,inf,...,inf,inf,299,inf,inf,inf,inf,inf,inf,inf
4,MX000017004,2010,02,TMIN,inf,144,inf,144,inf,inf,...,inf,inf,107,inf,inf,inf,inf,inf,inf,inf
5,MX000017004,2010,02,PRCP,inf,0,inf,0,inf,inf,...,0,inf,inf,0,inf,0,inf,inf,inf,inf
6,MX000017004,2010,03,TMAX,inf,inf,inf,inf,321,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
7,MX000017004,2010,03,TMIN,inf,inf,inf,inf,142,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
8,MX000017004,2010,03,PRCP,inf,0,inf,0,inf,0,...,0,inf,0,inf,0,inf,0,inf,inf,0
9,MX000017004,2010,04,TMAX,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,363,inf,inf,inf,inf


In [8]:
# Procedura opdowiadająca za zapisanie do pliku aktualnego dataframe'a
filtred_data_path = os.getcwd()[:-13] + "analysis_data/filtred_data.txt"
print(filtred_data_path)
with open(filtred_data_path, 'w') as out:
    out.write(df_not_tidied.to_csv(index=False))

/home/mbiela/Documents/AiBD/lab2/analysis_data/filtred_data.txt


#### Fragment kodu odpowiadający za utworzenie nowego DataFrame'a już wyczyszczonego z błednych lub nie pełnych danych

In [9]:
df_tidied = pd.melt(df_not_tidied, ['id', 'year', 'month', 'element'])
df_tidied = df_tidied[df_tidied.value != 'inf']
df_tidied = df_tidied.replace(to_replace=["day"],value="",regex=True)

df_tidied.head(20)

,id,year,month,element,variable,value
11,MX000017004,2010,04,PRCP,1,0
18,MX000017004,2010,02,TMAX,2,273
19,MX000017004,2010,02,TMIN,2,144
20,MX000017004,2010,02,PRCP,2,0
23,MX000017004,2010,03,PRCP,2,0
41,MX000017004,2010,04,PRCP,3,0
48,MX000017004,2010,02,TMAX,4,241
49,MX000017004,2010,02,TMIN,4,144
50,MX000017004,2010,02,PRCP,4,0
53,MX000017004,2010,03,PRCP,4,0


#### Nasz DataFrame nie spełnia jeszcze zasad tidy data dlatego
 - w tym celu połącze kolunny odpowiadające za rok, miesiac, dzien w jedną kolumnę o nazwie date.
 - usuwam już niepotrzebne kolumny tj.: year, variable, month
 - następnie aktualizuje nasz DataFrame do następujących kolumn id, date, element, value

In [10]:
df_tidied = df_tidied.assign(date = df_tidied.year.astype(str) + "-" + df_tidied.month.astype(str) + "-" + df_tidied.variable.astype(str)[:] )
df_tidied.pop("year")
df_tidied.pop("variable")
df_tidied.pop("month")

df_tidied = df_tidied[["id","date","element","value"]]

df_tidied.head(20)

,id,date,element,value
11,MX000017004,2010-04-1,PRCP,0
18,MX000017004,2010-02-2,TMAX,273
19,MX000017004,2010-02-2,TMIN,144
20,MX000017004,2010-02-2,PRCP,0
23,MX000017004,2010-03-2,PRCP,0
41,MX000017004,2010-04-3,PRCP,0
48,MX000017004,2010-02-4,TMAX,241
49,MX000017004,2010-02-4,TMIN,144
50,MX000017004,2010-02-4,PRCP,0
53,MX000017004,2010-03-4,PRCP,0


In [11]:
temporary_tidied_path = os.getcwd()[:-13] + "analysis_data/tidied_data.txt"
print(temporary_tidied_path)
with open(temporary_tidied_path, 'w') as file:
    file.write(df_tidied.to_csv(index=False))

/home/mbiela/Documents/AiBD/lab2/analysis_data/tidied_data.txt


#### Data Appendix
Dobrym sposobem na lepsze przedstwienie naszych danych będzie podzielenie kolumny element, która zawiera 3 typy pomiarów do osobnych kolumn kolejno: </br></br>
- TMAX
- TMIN
- PRCP

In [12]:
# Rozdzielenie TMAX, TMIN, PRCP do osobnych kolumn
df = pd.pivot_table(df_tidied, index=['id', 'date'], columns=['element'], values='value', aggfunc='last')
df.dropna(inplace=True)
df.reset_index(drop=False, inplace=True)


In [13]:
df.head()

element,id,date,PRCP,TMAX,TMIN
0,MX000017004,2010-02-2,0,273,144
1,MX000017004,2010-02-4,0,241,144
2,MX000017004,2010-04-27,0,363,167
3,MX000017004,2010-05-27,0,332,182


#### Zapis danych końcowych do pliku

In [14]:
final_tidied_path = os.getcwd()[:-13] + "analysis_data/final_data.txt"
print(temporary_tidied_path)
with open(final_tidied_path, 'w') as file:
    file.write(df.to_csv(index=False))

/home/mbiela/Documents/AiBD/lab2/analysis_data/tidied_data.txt


In [15]:
# Zrobienie kopii do Data Appendix
data_appendix_path = os.getcwd()[:-13] + "documents/data_appendix.txt"
with open(data_appendix_path, 'w') as file:
    file.write(df.to_csv(index=False))

## Wnioski
Laboratorium pozwoliło mi na zapoznanie się z: </br></br>
- Zasadami tidy data w procesie obrabiania zbioru danych
- Profesjonalną dokumentacją do zbioru danych